<a href="https://colab.research.google.com/github/stardust75R/test-google-colab/blob/main/signateFoundryVGG16_ipynb_%E3%81%AEsigmoid2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
dir_name = "/content/gdrive/My Drive/ColabData/signate2/foundry/"

In [ ]:
import pandas as pd
import os
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, roc_auc_score,classification_report
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
seed = 11235
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [ ]:
img_size1 = 96
img_size2 = 96     #画像のサイズ(今回は96*96)
channels = 128            #画像のチャンネル数を調整
max_epoch = 200         #学習のエポック数を調整、より多くの学習ができる
mini_batch_size = 256   #ミニバッチサイズを調整
dropout_rate = 0.8    #ドロップアウト率を調整。モデル表現力を低下させ、過学習を防ぐために用いられる
learning_rate = 0.001 #学習スピードを調整
lambda1 = 0.0001        #L1正則化の係数を調整。L1正則化は過学習防止に用いられる
lambda2 = 0.0001

print(os.listdir(dir_name))

['label_master.tsv', 'train_master.tsv', 'test_images', 'train_images(1)']


In [ ]:
df = pd.read_csv(os.path.join(dir_name,'train_master.tsv'), delimiter='\t')
print(df.head(5))

     file_name  label_id
0  train_0.jpg         1
1  train_1.jpg         5
2  train_2.jpg         1
3  train_3.jpg         6
4  train_4.jpg         3


In [ ]:
dir_name_train = 'train_images(1)/'

datagen = ImageDataGenerator(rotation_range=45,width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,horizontal_flip=False,fill_mode='bicubic')

imgarray = []
category = []

for filename in sorted(os.listdir(os.path.join(dir_name, dir_name_train))):
  img = keras.preprocessing.image.load_img(os.path.join(dir_name, dir_name_train, filename), color_mode="rgb", target_size=(img_size1, img_size2,3))
  imgdata = keras.preprocessing.image.img_to_array(img)
  imgdata = imgdata / 255.0
  imgarray.append(imgdata)
  temp = df[df['file_name']==filename].file_name
  label = df[df['file_name']==filename].label_id.values[0]
  category.append(label)

imgarray = np.array(imgarray)
category = np.array(category)

In [ ]:
x_train, x_valid, t_train, t_valid = train_test_split(imgarray, category, test_size=0.3, random_state=0)

print("x_train.shape = ", x_train.shape, "t_train.shape = ", t_train.shape)
print("x_valid.shape = ", x_valid.shape, "t_valid.shape = ", t_valid.shape)

x_train.shape =  (3500, 96, 96, 3) t_train.shape =  (3500,)
x_valid.shape =  (1500, 96, 96, 3) t_valid.shape =  (1500,)


In [ ]:
train_labels = keras.utils.to_categorical(t_train, num_classes=10)
valid_labels = keras.utils.to_categorical(t_valid, num_classes=10) #ワンホットエンコードする

In [ ]:
from keras.applications.vgg16 import VGG16

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(img_size1, img_size2,3))
conv_base.trainable = False
conv_base.summary()


network = keras.models.Sequential()

network.add(conv_base)

network.add(keras.layers.BatchNormalization())
network.add(keras.layers.Conv2D(filters=channels, kernel_size=(3,3), strides=(1,1), padding='same', activation='elu'))
network.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides=None, padding='same'))

network.add(keras.layers.Flatten())

network.add(keras.layers.Dropout(dropout_rate))
network.add(keras.layers.Dense(units=256, activation='relu', kernel_regularizer=keras.regularizers.l1_l2(l1=lambda1,l2=lambda2)))
network.add(keras.layers.Dense(units=10,  activation='softmax')) #変更したところ

opt = keras.optimizers.RMSprop(lr=learning_rate)
network.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

network.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 96, 96, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 96, 96, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 48, 48, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 48, 48, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 48, 48, 128)       147584    
                                                              

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 batch_normalization (Batch  (None, 3, 3, 512)         2048      
 Normalization)                                                  
                                                                 
 conv2d (Conv2D)             (None, 3, 3, 128)         589952    
                                                                 
 max_pooling2d (MaxPooling2  (None, 2, 2, 128)         0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0

In [ ]:
history = network.fit(x_train, train_labels, epochs=max_epoch, batch_size=mini_batch_size, validation_data=(x_valid, valid_labels))

Epoch 1/200
14/14 [==============================] - 619s 45s/step - loss: 2.7642 - accuracy: 0.4060 - val_loss: 2.1872 - val_accuracy: 0.4380
Epoch 2/200
 3/14 [=====>........................] - ETA: 5:33 - loss: 1.8528 - accuracy: 0.5807

In [ ]:
network.save('model.h5', include_optimizer=False)

In [ ]:
acc_train  = np.array(history.history['accuracy'])
acc_valid  = history.history['val_accuracy']
loss_train = history.history['loss']
loss_valid = history.history['val_loss']

In [ ]:
epochs = range(1, len(loss_train) + 1)
plt.plot(epochs, loss_train, 'bo', label='Training loss')
plt.plot(epochs, loss_valid, 'b',  label='Validatoin loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.yscale('log')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.plot(epochs, acc_train, 'bo', label='Training acc')
plt.plot(epochs, acc_valid, 'b',  label='Validation acc')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1.1])
plt.legend()
plt.grid()
plt.show()

In [ ]:
y_train_predict = network.predict(x_train)
t_train_predict = np.argmax(y_train_predict, axis=1)
print(confusion_matrix(t_train, t_train_predict))

In [ ]:
y_valid_predict = network.predict(x_valid)
t_valid_predict = np.argmax(y_valid_predict, axis=1)
print(confusion_matrix(t_valid, t_valid_predict))

In [ ]:
dir_name_test = os.path.join('test_images/')

imgarray = []
filenamearray = []

for filename in sorted(os.listdir(os.path.join(dir_name, dir_name_test))):
  img = keras.preprocessing.image.load_img(os.path.join(dir_name, dir_name_test, filename), color_mode="rgb", target_size=(img_size1, img_size2))
  imgdata = keras.preprocessing.image.img_to_array(img)
  imgdata = imgdata / 255.0
  imgarray.append(imgdata)
  filenamearray.append(filename)

x_test = np.array(imgarray)
print("x_test.shape = ", x_test.shape)

In [ ]:
y_test = network.predict(x_test)
t_test_predict = np.argmax(y_test,axis=1)

# ファイルのオープン
f = open('submit.tsv', 'w', encoding='utf-8')

# 1 行ずつ書き込み
for n in range(0, len(t_test_predict)):
  # image_id を文字列に変換
  image_id = str(filenamearray[n])

  # label を 0 から 9 の整数に変換
  label = int(t_test_predict[n])

  # 1 行に 2 つの列を書き込む
  f.write('{}\t{}\n'.format(image_id, label))

# ファイルのクローズ
f.close()